In [1]:
import langchain

In [2]:
# loading the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["google_api_key"] = os.getenv("GOOGLE_API_KEY")

### simple LLM call with streaming

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import SystemMessage, HumanMessage

In [8]:
model = init_chat_model(
    model="gemma2-9b-it",       # just the model name
    model_provider="groq",      # explicitly specify provider
    api_key= os.getenv("GROQ_API_KEY") # make sure GROQ_API_KEY is set
)


In [9]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001502B2BA7B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001502B2BB0E0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [12]:
## create messages
messages = [
    SystemMessage("you are a helpful assistant"),
    HumanMessage("what are the top 3 benefis of using Langchain??" )
]

In [13]:
## invoke the model
response = model.invoke(messages)

In [14]:
response

AIMessage(content="It's great you're interested in LangChain! \n\nAs a helpful assistant, I can highlight these **top 3 benefits** of using LangChain:\n\n1. **Simplified Development:** LangChain provides a modular and composable framework, making it much easier to build and deploy applications with large language models (LLMs). It offers pre-built components for tasks like prompting, memory management, and agent orchestration, saving you time and effort.\n\n2. **Flexibility and Extensibility:** LangChain is designed to be highly flexible. You can easily integrate various LLMs, data sources, and tools into your applications. This allows you to tailor your solutions to specific needs and experiment with different combinations.\n\n3. **Unlocking LLMs' Potential:** LangChain empowers developers to go beyond basic text generation and leverage the full capabilities of LLMs. It enables you to build applications that can interact with external systems, access and process information, and perfo

In [15]:
print(response.content)

It's great you're interested in LangChain! 

As a helpful assistant, I can highlight these **top 3 benefits** of using LangChain:

1. **Simplified Development:** LangChain provides a modular and composable framework, making it much easier to build and deploy applications with large language models (LLMs). It offers pre-built components for tasks like prompting, memory management, and agent orchestration, saving you time and effort.

2. **Flexibility and Extensibility:** LangChain is designed to be highly flexible. You can easily integrate various LLMs, data sources, and tools into your applications. This allows you to tailor your solutions to specific needs and experiment with different combinations.

3. **Unlocking LLMs' Potential:** LangChain empowers developers to go beyond basic text generation and leverage the full capabilities of LLMs. It enables you to build applications that can interact with external systems, access and process information, and perform complex reasoning tasks.

In [16]:
# streaming response

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

As a helpful assistant, I can tell you that LangChain is a powerful tool with many benefits. Here are three of the top ones:

1. **Simplifies building complex applications with LLMs:** LangChain provides a framework and building blocks to easily connect different components like LLMs, databases, and APIs. This makes it much easier to build sophisticated applications that leverage the power of large language models without needing to write a lot of boilerplate code.
2. **Enhances LLM capabilities:** LangChain offers tools for memory management, prompting, chain creation, and more. These tools help you go beyond basic text generation and build LLMs that can understand context, remember past interactions, and perform complex reasoning tasks.
3. **Promotes modularity and reusability:** LangChain encourages a modular approach to LLM development. You can easily create reusable components like prompts, chains, and agents that can be combined and customized for different applications. This sav

In [18]:
## Dynamic propts

from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "You are a helpful assistant"),
    ("user", "Tell me a joke about {topic}")
])

prompts = prompt_template.invoke({"topic": "cats"})

In [20]:
res = model.invoke(prompts)
print(res.content)

Why don't cats play poker in the jungle? 

Too many cheetahs! 😹  



In [21]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# Create a more complex chain
def create_story_chain():
    # Template for story generation
    story_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a creative storyteller. Write a short, engaging story based on the given theme."),
        ("user", "Theme: {theme}\nMain character: {character}\nSetting: {setting}")
    ])
    
    # Template for story analysis
    analysis_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a literary critic. Analyze the following story and provide insights."),
        ("user", "{story}")
    ])
    
    # Build the chain - Method 1: Sequential execution
    story_chain = (
        story_prompt 
        | model 
        | StrOutputParser()
    )
    
    # Create a function to pass the story to analysis
    def analyze_story(story_text):
        return {"story": story_text}
    
    analysis_chain = (
        story_chain
        | RunnableLambda(analyze_story)
        | analysis_prompt
        | model
        | StrOutputParser()
    )
    return analysis_chain

In [22]:
chain=create_story_chain()
chain

ChatPromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a creative storyteller. Write a short, engaging story based on the given theme.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['character', 'setting', 'theme'], input_types={}, partial_variables={}, template='Theme: {theme}\nMain character: {character}\nSetting: {setting}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001502B2BA7B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001502B2BB0E0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(analyze_story)
| ChatPromptTemplate(input_variables=['story'], input_types={}, partial_variables={}, mes

In [23]:
result = chain.invoke({
    "theme": "artificial intelligence",
    "character": "a curious robot",
    "setting": "a futuristic city"
})

print("Story and Analysis:")
print(result)

Story and Analysis:
This short story is a charming exploration of artificial intelligence and the nature of human experience. 

**Themes:**

* **The pursuit of knowledge vs. emotional understanding:** Bolt, designed to learn and analyze, initially focuses on the objective data of the marketplace. His encounter with the Dream Weaver forces him to confront the subjective realm of emotions and dreams, realizing that true understanding goes beyond cold, hard facts.
* **The power of human connection:** The interaction between Bolt and the girl highlights the importance of empathy and shared experiences.  Bolt's newfound understanding of "happiness" comes not from processing data, but from connecting with the girl's genuine joy.
* **The nature of consciousness:** Through Bolt's experience with the Dream Weaver, the story touches on the elusive nature of consciousness. While Bolt can process vast amounts of information, he lacks the inherent human capacity for emotional experience and dreamin